In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
pd.options.plotting.backend = "matplotlib"
template = "plotly_dark"

In [ ]:
# %%
def load_price_timeseries(file: str) -> pd.Series:
    df = pd.read_csv(file)
    df.index = pd.to_datetime(df["Date"], format="%d/%m/%Y %H:%M")
    return df["Intraday Continuous 15 minutes ID1-Price"]  # * 1e-6 # €/MWh -> €/Wh


In [ ]:
def calc_throughput(df):
    power = df["power_sim"]
    power_pos = power[power > 0].sum() * 0.25
    power_neg = power[power < 0].abs().sum() * 0.25
    return (power_pos + power_neg) / 2 / 180e3

In [ ]:
def calc_roundtrip_efficiency(res):
    p = res["power_sim"]
    e_pos = p[p > 0].abs().sum() * 0.25  # Wh
    e_neg = p[p < 0].abs().sum() * 0.25  # Wh

    delta_soc = res["soc_sim"].iloc[-1] - res["soc_sim"].iloc[0]
    delta_e = delta_soc * 180e3  # Wh

    return np.abs(e_neg) / (e_pos - delta_e)

In [ ]:
def calc_revenue(res, price):
    df = res.join(price)
    return -1 * sum(df["power_sim"] * df["Intraday Continuous 15 minutes ID1-Price"]) * 0.25 * 1e-6  # W -> MWh

In [ ]:
price = load_price_timeseries("data/intraday_prices/electricity_prices_germany_2021.csv")
# price.index.name = "time"

In [ ]:
df_lp95 = pd.read_parquet("results/eff95/2021 LP r=1.0 fec=2.0.parquet", engine="pyarrow")
df_lp96 = pd.read_parquet("results/2021 LP r=1.0 fec=2.0.parquet", engine="pyarrow")
df_nl = pd.read_parquet("results/eff95/2021 NL r=1.0 fec=2.0.parquet", engine="pyarrow")

In [ ]:
l_batt_nl = float(df_nl["battery_losses"].sum()) * 0.25 * 1e-3
l_conv_nl = float(df_nl["converter_losses"].sum()) * 0.25 * 1e-3
print(f"{l_batt_nl=:.2f}")
print(f"{l_conv_nl=:.2f}")
print(f"{l_batt_nl + l_conv_nl:.2f}")

In [ ]:
l_batt_lp = float(df_lp["battery_losses"].sum()) * 0.25 * 1e-3
l_conv_lp = float(df_lp["converter_losses"].sum()) * 0.25 * 1e-3
print(f"{l_batt_lp=:.2f}")
print(f"{l_conv_lp=:.2f}")
print(f"{l_batt_lp + l_conv_lp:.2f}")

In [ ]:
df_lp = df_lp95

fec_lp = calc_throughput(df_lp)
fec_nl = calc_throughput(df_nl)
print(f"{fec_lp=:.1f}")
print(f"{fec_nl=:.1f} ({fec_nl/fec_lp - 1:+.2%})")
print("")

eff_lp = calc_roundtrip_efficiency(df_lp)
eff_nl = calc_roundtrip_efficiency(df_nl)
print(f"{eff_lp=:.1%}")
print(f"{eff_nl=:.1%} ({(eff_nl - eff_lp):+.2%})")
print("")

rev_lp = calc_revenue(df_lp, price)
rev_nl = calc_revenue(df_nl, price)
print(f"{rev_lp=:.1f} €")
print(f"{rev_nl=:.1f} € ({rev_nl/rev_lp - 1:+.2%})")

In [ ]:
df_lp = df_lp96

fec_lp = calc_throughput(df_lp)
fec_nl = calc_throughput(df_nl)
print(f"{fec_lp=:.1f}")
print(f"{fec_nl=:.1f} ({fec_nl/fec_lp - 1:+.2%})")
print("")

eff_lp = calc_roundtrip_efficiency(df_lp)
eff_nl = calc_roundtrip_efficiency(df_nl)
print(f"{eff_lp=:.1%}")
print(f"{eff_nl=:.1%} ({(eff_nl - eff_lp):+.2%})")
print("")

rev_lp = calc_revenue(df_lp, price)
rev_nl = calc_revenue(df_nl, price)
print(f"{rev_lp=:.1f} €")
print(f"{rev_nl=:.1f} € ({rev_nl/rev_lp - 1:+.2%})")
print("")

In [ ]:
def plot_timeseries(df, **kwargs):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    fig.update_layout(**kwargs)

    for id in ["soc_sim", "soc_opt"]:
        fig.add_trace(go.Scatter(x=df.index, y=df[id], name=id), row=1, col=1)

    for id in ["power_sim", "power_opt"]:
        fig.add_trace(go.Scatter(x=df.index, y=df[id], name=id), row=2, col=1)
    
    return fig


In [ ]:
plot_timeseries(df_nl, height=800, template=template)

In [ ]:
plot_timeseries(df_lp, height=800, template=template)

In [ ]:
def calc_roundtrip_efficiency(res):
    p = res["power_sim"]
    e_pos = p[p > 0].abs().sum()  # Wh
    l_pos = res.loc[p > 0, "converter_losses"].sum() + res.loc[p > 0, "battery_losses"].sum()
    eff_pos = (e_pos - l_pos) / (e_pos)
    
    e_neg = p[p < 0].abs().sum()  # Wh
    l_neg = res.loc[p < 0, "converter_losses"].sum() + res.loc[p < 0, "battery_losses"].sum()
    eff_neg = (e_neg - l_neg) / (e_neg)
    # e_neg = p[p < 0].abs().sum() * 0.25  # Wh

    # delta_soc = res["soc_sim"].iloc[-1] - res["soc_sim"].iloc[0]
    # delta_e = delta_soc * 180e3  # Wh

    # return np.abs(e_neg) / (e_pos - delta_e)
    return eff_pos * eff_neg

In [ ]:
eff_lp = calc_roundtrip_efficiency(df_lp)
eff_nl = calc_roundtrip_efficiency(df_nl)
print(f"{eff_lp=:.2%}")
print(f"{eff_nl=:.2%} ({(eff_nl - eff_lp):+.2%})")